# Radiosité avec occlusions

On commence par charger les fonctions de base

In [1]:
import numpy as np
import random

DEFAULT_RHO = 0.55
print(f"default reflectance rho value = {DEFAULT_RHO}")

# Representation
import pandas as pd
from IPython.display import display, HTML
print("imports: OK")

# Display Array, descending y axis
def dumpdata(msg, data):
    print(f"## {msg} ##\n {pd.DataFrame(data).sort_index(ascending=False, axis=0)}\n")

def color_max_red(value):
    v = int(value)
    hexcolor = '#%02x%02x%02x' % (v, v, v)
    return 'background-color: {}'.format(hexcolor)

def convertRatioToRGB(source):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (source[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(source[y, x]) * 255)
                
def fakeDisplay(msg="", style=True):
    df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
    
    if style:
        s = df.style.applymap(color_max_red)
    else:
        s = df.style
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

def fdisplay(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=False, axis=0)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(df.style.render()+"</br>"))

def fdisplayTrans(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=True, axis=0).transpose()
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(df.style.render()+"</br>"))

default reflectance rho value = 0.55
imports: OK


## Modifications et ajouts

### Classe Rect

- init: On a besoin des 4 points pour construire la surface et de déterminer son vecteur normal (attention à l'ordre des points)
- prepareInter: définit les arrêtes du rectangle dont on se servira pour tester les intersections
- intersect: le point d'entrée pour déterminer si un rayon a une intersection avec le rectangle. Chaque rectangle est divisé en 2 triangles
- intersectTriangle: test intersection entre un rayon et un triangle (note: en fait on limite le test au segment [xi;xj] porté par rij pour éliminer les intersections qui se trouvent après la cible). J'utilise l'algorithme proposé dans T.Möller et B. Trumbore [Fast, Minimum Storage Ray/Triangle Intersection](https://cadxfem.org/inf/Fast%20MinimumStorage%20RayTriangle%20Intersection.pdf)

### Classe Mesh

- initObstacles: on établit la liste des obstacles. Seuls les murs verticaux sont pris en compte
- findObstacles: recherche les surfaces qui peuvent potentiellement bloquer un rayon entre une source et une cible
- hasOcclusion: test si un rayon donné a un point d'intersection avec une surface entre une source et une cible
- getFormFactor_uni_sq: modifiée pour appeler _hasOcclusion_

### Général

- ajout d'un mur (parallèle au plan x0z)
    

In [2]:
class Rect():
    count = -1
    EPSILON = 0.00001

    def next():
        Rect.count += 1
        return Rect.count

    def __init__(self, v0, v1, v2, v3, rho=DEFAULT_RHO, E=0.):
        self.v0 = v0
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3

        # TODO: retro compatibility
        self.cmin = v0
        self.cmax = v2

        self.idt = Rect.next()
        # Radiosity
        self.rho = rho
        self.E = E
        self.B = E
        self.dB = E

        # Geometry
        self.lx = self.cmax[0] - self.cmin[0]
        self.ly = self.cmax[1] - self.cmin[1]
        self.lz = self.cmax[2] - self.cmin[2]
        self.area = self.lx*self.ly + self.lx*self.lz + self.ly*self.lz

        # Patchs
        self.split()

        # Intersect
        self.prepareInter()
        self.norm = np.cross(self.edge0a, self.edge0b)
        n = np.linalg.norm(self.norm)
        self.norm /= n
        print(f"{self.idt} {self.cmin} {self.cmax} area:{self.area} norm:{self.norm}")

    def prepareInter(self):
        self.edge0a = self.v1 - self.v0
        self.edge0b = self.v3 - self.v0
        self.edge2a = self.v3 - self.v2
        self.edge2b = self.v1 - self.v2
    
    def intersect(self, orig, d):
        if self.intersectTriangle(orig, d, self.v0, self.v1, self.v3, self.edge0a, self.edge0b):
            return True
        if self.intersectTriangle(orig, d, self.v2, self.v1, self.v3, self.edge2a, self.edge2b):
            return True
        return False
    
    def intersectTriangle(self, orig, d, v0, v1, v2, edge1, edge2):
        #print(f"o:{orig} d:{d} v0{v0} v1{v1} v2{v2} e1{edge1} e2{edge2}")
        #pvect = np.cross(d, edge2)
        pvect = np.array([0., 0., 0.])
        pvect[0] = d[1]*edge2[2] - d[2]*edge2[1]
        pvect[1] = d[2]*edge2[0] - d[0]*edge2[2]
        pvect[2] = d[0]*edge2[1] - d[1]*edge2[0]
        
        det = edge1[0]*pvect[0] + edge1[1]*pvect[1] + edge1[2]*pvect[2]

        if det < Rect.EPSILON:
            #print("back face")
            return False

        tvect = orig - v0
        u = tvect[0]*pvect[0] + tvect[1]*pvect[1] + tvect[2]*pvect[2]

        if u < 0. or u > det:
            #print(f"u<0 or u>det (u:{u}, det:{det}")
            return False

        #qvect = np.cross(tvect, edge1)
        qvect = np.array([0., 0., 0.])
        qvect[0] = tvect[1]*edge1[2] - tvect[2]*edge1[1]
        qvect[1] = tvect[2]*edge1[0] - tvect[0]*edge1[2]
        qvect[2] = tvect[0]*edge1[1] - tvect[1]*edge1[0]
        
        v = d[0]*qvect[0] + d[1]*qvect[1] + d[2]*qvect[2]
        if v < 0. or u+v > det:
            #print(f"v<0 or u+v>det (u:{u}, v:{v}, u+v:{u+v}, det:{det}")
            return False
        t = edge2[0]*qvect[0] + edge2[1]*qvect[1] + edge2[2]*qvect[2]
        inv_det = 1.0 / det
        t *= inv_det

        if t > 1.:
            return False
        
        return True
        
    def initLightSource(self, E, rho=0.):
        self.E = E
        self.B = E
        self.dB = E
        self.rho = rho
    
    def resetRadiosity(self, rho=DEFAULT_RHO):
        self.B = self.E
        self.dB = self.E
        self.rho = rho

    def getRndUV(self, umin, umax, vmin, vmax):
        # TODO: p have to be in ]0;1[ to avoid the r = 0 case (in Ri T Rj)
        pu = rng.uniform(umin, umax)
        pv = rng.uniform(vmin, vmax)
        #print(f"[get2Rnd] u({umin},{umax}) v({vmin}, {vmax}) {pu}, {pv}")
        return (pu, pv)

    def getRandomPoint(self):
        if (self.lz == 0):
            # R xOy
            (px, py) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[1], self.cmax[1])
            pz = self.cmin[2]
        elif (self.ly == 0):
            # R xOz
            (px, pz) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[2], self.cmax[2])
            py = self.cmin[1]
        else:
            # R yOz
            (py, pz) = self.getRndUV(self.cmin[1], self.cmax[1], self.cmin[2], self.cmax[2])
            px = self.cmin[0]
        #print(f"(idt:{self.idt} return point ({px}, {py}, {pz})");
        return(np.array([px, py, pz]))

    def getPatchs(self):
        return self.patch

    def split(self, dx=0.5, dy=0.5, dz=0.5):
        self.patch = []

        # Hack to ensure we do at least 1 iteration
        lx = dx  if self.lx == 0 else self.lx
        ly = dy  if self.ly == 0 else self.ly
        lz = dz  if self.lz == 0 else self.lz

        # used to cancel splits on u, v, w
        kx = 0. if lx < 1. else 1.
        ky = 0. if ly < 1. else 1.
        kz = 0. if lz < 1. else 1.

        x0 = self.cmin[0]
        y0 = self.cmin[1]
        z0 = self.cmin[2]

        z = dz / 2.
        while z < lz:
            y = dy / 2.
            while y < ly:
                x = dx / 2.
                while x < lx:
                    #print(f"{x0+x*kx}, {y0+y*ky}, {z0+z*kz}")
                    self.patch.append(np.array([x0+x*kx, y0+y*ky, z0+z*kz]))
                    x += dx
                y += dy
            z += dz

    def initFormFactors(self, n):
        self.fij = np.zeros([n])

    def dumpFij(self):
        n = len(self.fij)
        for i in range(n):
            print(f"[{self.idt} -> {i}] Fij = {self.fij[i]}")
            
print("class Rectangle: OK")

class Mesh():
    def __init__(self):
        self.poly = []
        # Radiosity
        self.maxUnshotIndex = -1

    def add(self, item):
        self.poly.append(item)

    def get(self, index):
        return self.poly[index]

    def resetRadiosity(self, rho=DEFAULT_RHO):
        for p in self.poly:
            p.resetRadiosity()
    
    def resetLights(self):
        for p in self.poly:
            p.E = 0.
            p.B = p.E
            p.dE = p.E
    
    def initRadiosity(self):
        # find poly with max E
        maxUnshot = 0.
        for p in self.poly:
            unshot = p.dB * p.area
            if  unshot > maxUnshot:
                maxUnshot = unshot
                self.maxUnshotIndex = p.idt
        print(f"[initRadiosity] maxUnshotIndex = {self.maxUnshotIndex}")

    def solveRadiosity(self, epsilon=1., n=100):
        step = 0.
        sumdRad = epsilon
        while sumdRad >= epsilon and step < n :
            src = self.get(self.maxUnshotIndex)
            maxUnshot = 0.
            sumdRad = 0.
            for rec in self.poly:
                if rec.idt != src.idt:
                    dRad = src.dB * rec.rho * rec.fij[src.idt]
                    rec.dB += dRad
                    rec.B += dRad
                    tmp_unshot = rec.dB * rec.area
                    sumdRad += dRad
                    if tmp_unshot > maxUnshot:
                        self.maxUnshotIndex = rec.idt
                        maxUnshot = tmp_unshot
            src.dB = 0
            step += 1
        print(f"[{step}] maxUnshotIndex={self.maxUnshotIndex} dB={src.dB} sum dRad = {sumdRad}")

    def showAllBi(self):
        for p in self.poly:
            print(f"B{p.idt} = {p.B}")
            
    def initObstacles(self, width, height):
        # 2D grid to track vertical walls
        # +1 for the farthest walls
        self.gridobst = [[ [] for x in range(width+1)] for y in range(height+1)]

        for p in self.poly:
            # only vertical surfaces
            if p.norm[2] == 0.:
                # use cmin as index
                x = int(p.cmin[0])
                y = int(p.cmin[1])
                self.gridobst[x][y].append(p)

    def findObstacles(self, ri, rj):
        obst = []
        # cmin as index like gridobst
        xmin = int(min(ri.cmin[0], rj.cmin[0]))
        xmax = int(max(ri.cmin[0], rj.cmin[0]))
        ymin = int(min(ri.cmin[1], rj.cmin[1]))
        ymax = int(max(ri.cmin[1], rj.cmin[1]))
        #print(f"({xmin} - {xmax}) , ({ymin} - {ymax})")
        for y in range(ymin, ymax+1):
            for x in range(xmin, xmax+1):
                l = self.gridobst[x][y]
                if len(l) != 0:
                    obst += l
        # TODO: cost?
        if ri in obst:
            obst.remove(ri)
        if rj in obst:
            obst.remove(rj)
        #for p in obst:
        #    print(f"{p.idt}")
        #print(f"obstacles_ij len = {len(obst)}")
        return obst

    def hasOcclusion(self, xi, xj, rij, obstacles_ij, ri, rj):
        for p in obstacles_ij:
            #if p.idt != i and p.idt != j:
            if p.intersect(xi, rij):
                #print(f"INTERSECTION i:{xi} d:{rij} -> j:{xj} with {p.idt} ; [ri:{ri.idt} rj:{rj.idt}]")
                return True
        return False

    def initFormFactors(self):
        size = len(self.poly)
        for p in self.poly:
            p.initFormFactors(len(self.poly))

    def getFormFactors_uni_sq2(self):
        count = len(self.poly)
        for i in range(count-1):
            poly_i = self.poly[i]
            for j in range(i+1, count):
                poly_j = self.poly[j]
                fij = self.getFormFactor_uni_sq(poly_i, poly_j)
                poly_i.fij[poly_j.idt] = fij
                poly_j.fij[poly_i.idt] = fij * poly_i.area / poly_j.area
                    
    def getFormFactor_uni_sq(self, ri, rj):
        #print(f"getFF {ri.idt} -> {rj.idt}")
        fij = 0.
        aj = rj.area
        patchs_i = ri.getPatchs()
        patchs_j = rj.getPatchs()

        ui = patchs_i[0]
        uj = patchs_j[0]
        ux = np.array([1., 0., 0.])
        m = np.stack((ui, uj, ux))
        if np.linalg.det(m) == 0.:
            return fij      
        
        li = len(patchs_i)
        lj = len(patchs_j)
        samples = li * lj
        
        obstacles_ij = self.findObstacles(ri, rj)
        
        for xi in patchs_i:
            for xj in patchs_j:
                # get ray from r1 to r2
                rij = xj - xi
                #if True:
                if not self.hasOcclusion(xi, xj, rij, obstacles_ij, ri, rj):
                    r = np.linalg.norm(rij)
                    nrij = rij / r
                    r2 = r*r
                    costhetai = np.dot(ri.norm, nrij)
                    costhetaj = -1.*np.dot(rj.norm, nrij)
                    deltaf = ((costhetai * costhetaj) / (np.pi * r2 + aj/lj))
                    if (deltaf > 0.):
                        fij += deltaf
        fij = fij * aj/samples
        return fij

    def showFij(self, i):
        self.poly[i].dumpFij()

print("class Mesh: OK")

nz_up = np.array([0., 0., 1.])
nz_down = np.array([0., 0., -1.])

ny_up = np.array([0., 1., 0.])
ny_down = np.array([0., -1., 0.])

nx_up = np.array([1., 0., 0.])
nx_down = np.array([-1., 0., 0.])
print("normal vectors: OK")

c = 2.
WIDTH = 10
HEIGHT = 10
Rect.count = -1

mesh = Mesh()

# A word about point order in right-handed 3D coordinates
# if rectangle is
# d c
# a b
# norm > 0 (xOy, yOz), norm < 0 (x0z)

# Construct Mesh
## 1. Floor
print("## FLOOR ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, 0.]), np.array([x+1., y+1., 0.]), np.array([x, y+1, 0.])))

## 1. Ceiling
print("## CEILING ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, c]), np.array([x, y+1, c]), np.array([x+1., y+1., c]),  np.array([x+1, y, c])))

# South Wall
print("## SOUTH WALL ##")
y = 0
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y, c]), np.array([x+1, y, c]), np.array([x+1, y, 0])))

# North Wall
print("## NORTH WALL ##")
y = HEIGHT
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, 0]),  np.array([x+1, y, c]), np.array([x, y, c])))

# West Wall
print("## WEST WALL ##")
x = 0
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y+1, 0]), np.array([x, y+1, c]), np.array([x, y, c])))

# West Wall
print("## EAST WALL ##")
x = WIDTH
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y, c]),  np.array([x, y+1, c]), np.array([x, y+1, 0])))
    
print("mesh: OK")

y = 3
xmin = 1
xmax = 4
for x in range(xmin, xmax):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, 0]),  np.array([x+1, y, c]), np.array([x, y, c])))
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y, c]), np.array([x+1, y, c]), np.array([x+1, y, 0])))

print("obstacle: OK")

B_floor = np.zeros([HEIGHT, WIDTH])
B_ceiling = np.zeros([HEIGHT, WIDTH])
B_NWall = np.zeros([WIDTH])
B_SWall = np.zeros([WIDTH])
B_EWall = np.zeros([HEIGHT])
B_WWall = np.zeros([HEIGHT])

lighting_color = np.zeros([HEIGHT, WIDTH])

def polyToArrays():
    floorSize = (HEIGHT*WIDTH)
    idxCeiling = floorSize
    idxSWall = 2*floorSize
    idxNWall = idxSWall + WIDTH
    idxWWall = idxNWall + WIDTH
    idxEWall = idxWWall + HEIGHT
    endEWall = idxEWall + HEIGHT
    B_floor.fill(0.)
    B_ceiling.fill(0.)
    B_NWall.fill(0.)
    B_SWall.fill(0.)
    B_WWall.fill(0.)
    B_EWall.fill(0.)
    for p in mesh.poly:
        if p.idt >= 0 and p.idt < idxCeiling:
            x = p.idt % WIDTH
            y = int(p.idt / WIDTH)
            B_floor[y, x] = p.B
        elif p.idt >= idxCeiling and p.idt < idxSWall:
            x = (p.idt - idxCeiling) % WIDTH
            y = int((p.idt - idxCeiling) / WIDTH)            
            B_ceiling[y, x] = p.B
        elif p.idt >= idxSWall and p.idt < idxNWall:
            x = (p.idt - idxSWall)
            B_SWall[x] = p.B
        elif p.idt >= idxNWall and p.idt < idxWWall:
            x = (p.idt - idxNWall)
            B_NWall[x] = p.B
        elif p.idt >= idxWWall and p.idt < idxEWall:
            y = (p.idt - idxWWall)
            B_WWall[y] = p.B
        elif p.idt >= idxEWall and p.idt < endEWall:
            y = (p.idt - idxEWall)
            B_EWall[y] = p.B

class Rectangle: OK
class Mesh: OK
normal vectors: OK
## FLOOR ##
0 [0. 0. 0.] [1. 1. 0.] area:1.0 norm:[0. 0. 1.]
1 [1. 0. 0.] [2. 1. 0.] area:1.0 norm:[0. 0. 1.]
2 [2. 0. 0.] [3. 1. 0.] area:1.0 norm:[0. 0. 1.]
3 [3. 0. 0.] [4. 1. 0.] area:1.0 norm:[0. 0. 1.]
4 [4. 0. 0.] [5. 1. 0.] area:1.0 norm:[0. 0. 1.]
5 [5. 0. 0.] [6. 1. 0.] area:1.0 norm:[0. 0. 1.]
6 [6. 0. 0.] [7. 1. 0.] area:1.0 norm:[0. 0. 1.]
7 [7. 0. 0.] [8. 1. 0.] area:1.0 norm:[0. 0. 1.]
8 [8. 0. 0.] [9. 1. 0.] area:1.0 norm:[0. 0. 1.]
9 [9. 0. 0.] [10.  1.  0.] area:1.0 norm:[0. 0. 1.]
10 [0. 1. 0.] [1. 2. 0.] area:1.0 norm:[0. 0. 1.]
11 [1. 1. 0.] [2. 2. 0.] area:1.0 norm:[0. 0. 1.]
12 [2. 1. 0.] [3. 2. 0.] area:1.0 norm:[0. 0. 1.]
13 [3. 1. 0.] [4. 2. 0.] area:1.0 norm:[0. 0. 1.]
14 [4. 1. 0.] [5. 2. 0.] area:1.0 norm:[0. 0. 1.]
15 [5. 1. 0.] [6. 2. 0.] area:1.0 norm:[0. 0. 1.]
16 [6. 1. 0.] [7. 2. 0.] area:1.0 norm:[0. 0. 1.]
17 [7. 1. 0.] [8. 2. 0.] area:1.0 norm:[0. 0. 1.]
18 [8. 1. 0.] [9. 2. 0.] area:1.0 norm:[0

## Calcul des facteurs de forme

In [3]:
mesh.initObstacles(WIDTH, HEIGHT)
mesh.initFormFactors()
mesh.getFormFactors_uni_sq2()
print("Form Factors: OK")
#mesh.showFij(1)

Form Factors: OK


## Radiosité

In [4]:
mesh.resetLights()


#light = mesh.get(index) 
#light.initLightSource(31.830, 0.)
# Examples in cd/m^2
# Office (Accounting) 538
# Office              323
# Operating Room    19375
# Twilight             11
# Deep Twilight         1.07
# Moonlight             0.04
# Dusk                107
# Overcast day       1076
# Full dayligh      10764
# Direct sunlight  107639
intensity = 323

#light = mesh.get(113) 
light = mesh.get(122) 
#light = mesh.get(32) ## 5x5
light.initLightSource(intensity, 0.)
#light = mesh.get(112)
#light.initLightSource(500, 0.)
print("light source(s) : OK")

mesh.resetRadiosity()
mesh.initRadiosity()
mesh.solveRadiosity(intensity/1000, 500)
print("Radiance : OK")

#polyToArrays()
#print("Poly to arrays : OK")

light source(s) : OK
[initRadiosity] maxUnshotIndex = 122
[77.0] maxUnshotIndex=202 dB=0 sum dRad = 0.3163457904024139
Radiance : OK


## ... Et Affichage

In [5]:
polyToArrays()
#pd.DataFrame(B_NWall).transpose()
fdisplay(B_floor, "Floor")
fdisplay(B_ceiling, "Ceiling")
fdisplayTrans(B_SWall, "South Wall (!Watch out, y axis reverse)")
fdisplayTrans(B_NWall, "North Wall(!Watch out, y axis reverse)")
fdisplay(B_WWall, "West Wall")
fdisplay(B_EWall, "East Wall")

# http://eta-publications.lbl.gov/sites/default/files/lbl-35252.pdf
# A Contrast-based scalefactor for luminance display, G. Ward 1994
lwa = 0.
count = 0
for p in mesh.poly:
    if p.B != 0. and p.rho != -1. :
        lwa += np.log10(p.B)
        count += 1
lwa /= count
lwa += 0.84
lwa = np.power(10, lwa)

print(f"lwa = {lwa}")
ldmax = 150.

gamma = 2.2
sf = (1/ldmax) * np.power((1.219 + np.power(ldmax/2., 0.4)) / (1.219 + np.power(lwa, 0.4)), 2.5)

B_floor_sf = np.zeros([HEIGHT, WIDTH])
for y in range(HEIGHT):
    for x in range(WIDTH):
        B_floor_sf[y, x] = np.power(B_floor[y, x] * sf, 1/gamma)

fdisplay(B_floor_sf, "Floor sf")
data = np.copy(B_floor_sf)
convertRatioToRGB(data)
fakeDisplay("testing...", True)
print(f"lwa: {lwa} ; sf: {sf}")

,0,1,2,3,4,5,6,7,8,9
9,0.00393877,0.00460902,0.00517235,0.0057801,0.00613108,0.00651923,0.00641439,0.00623117,0.00586953,0.00518955
8,0.00590009,0.00712475,0.00797976,0.00854375,0.00887714,0.00911848,0.00944452,0.00897724,0.00768297,0.00657453
7,0.00968374,0.0119339,0.0120329,0.0130323,0.0135268,0.0145026,0.0144144,0.0129446,0.0108502,0.00907493
6,0.0174885,0.0218066,0.0199466,0.0205884,0.0210213,0.0245422,0.0230759,0.0197324,0.0161907,0.0151622
5,0.0421759,0.0417191,0.0339237,0.030232,0.0392854,0.0462353,0.0410516,0.032288,0.0419635,0.0353464
4,0.127592,0.0938231,0.0435777,0.0456087,0.0866506,0.0994309,0.137294,0.131683,0.0900836,0.0554455
3,0.769511,0.139144,0.0285258,0.0339933,0.527223,0.677887,0.542544,0.27499,0.16975,0.101731
2,5.2124,11.3125,16.6955,11.0614,4.6079,1.87985,0.835856,0.398904,0.20635,0.115544
1,4.73936,8.46153,11.0741,8.20271,4.11315,1.87725,0.869171,0.412911,0.209399,0.116148
0,3.31981,4.85236,5.68532,4.64666,2.85852,1.51266,0.768404,0.356413,0.184011,0.105452


,0,1,2,3,4,5,6,7,8,9
9,0.00564462,0.00642052,0.00705372,0.00768171,0.00804059,0.00865406,0.00880797,0.00846028,0.00791579,0.0069622
8,0.00863949,0.00999051,0.0108495,0.0116014,0.0120222,0.0128405,0.0134221,0.0126776,0.0109296,0.00931053
7,0.0146204,0.0167407,0.0167706,0.0182609,0.0193449,0.0214668,0.0216317,0.0193811,0.0162307,0.0138602
6,0.0265745,0.0301881,0.0274455,0.0299663,0.0321453,0.0390579,0.0371784,0.0316822,0.0264264,0.0194997
5,0.0619331,0.056195,0.0468359,0.0470635,0.0637842,0.0802411,0.0716929,0.0577115,0.0406577,0.0279324
4,0.185574,0.119712,0.0563946,0.0778665,0.155256,0.187567,0.15491,0.100567,0.0624725,0.0383031
3,0.713352,0.156247,0.03501,0.0620027,0.491349,0.496505,0.309362,0.171242,0.0929423,0.0517
2,2.38681,3.99106,328.749,3.73641,1.80309,0.94162,0.49719,0.246763,0.119731,0.061862
1,2.67079,3.52547,3.97445,3.24904,2.05061,1.13909,0.590608,0.28129,0.130608,0.0657994
0,2.26854,2.85047,3.07116,2.62175,1.80404,1.04163,0.552738,0.241991,0.113823,0.0600284


,0,1,2,3,4,5,6,7,8,9
0,3.1557,4.50882,5.17246,4.23234,2.67488,1.47571,0.707277,0.355529,0.192373,0.111926


,0,1,2,3,4,5,6,7,8,9
0,0.028014,0.0315331,0.0342534,0.0393373,0.0432666,0.0438288,0.0462526,0.0468673,0.0454291,0.0404903


,0
9,0.00473553
8,0.00626947
7,0.00906873
6,0.0165494
5,0.0396762
4,0.115793
3,1.01984
2,4.15354
1,4.16823
0,3.04424


,0
9,0.0299714
8,0.0412076
7,0.0539329
6,0.0822721
5,0.125404
4,0.169763
3,0.266136
2,0.300282
1,0.29968
0,0.285103


lwa = 0.70417209858352


,0,1,2,3,4,5,6,7,8,9
9,0.0318874,0.0342485,0.0360915,0.0379608,0.0389917,0.040095,0.0398006,0.0392798,0.0382266,0.036146
8,0.038317,0.0417468,0.0439538,0.0453396,0.0461354,0.0467013,0.0474531,0.0463711,0.043203,0.0402492
7,0.0479957,0.0527774,0.0529759,0.0549323,0.0558702,0.0576674,0.0575078,0.0547642,0.0505423,0.0465998
6,0.0627898,0.0694145,0.0666577,0.0676242,0.0682668,0.0732454,0.0712228,0.0663314,0.0606271,0.0588452
5,0.0936843,0.0932217,0.0848564,0.0805268,0.0907092,0.0976804,0.0925407,0.0829715,0.0934695,0.0864559
4,0.154951,0.134743,0.095087,0.0970764,0.129959,0.138346,0.160199,0.15719,0.132275,0.106088
3,0.350685,0.161177,0.0784283,0.0849354,0.295305,0.331048,0.299176,0.219675,0.176421,0.139791
2,0.836688,1.18995,1.42025,1.17787,0.791097,0.526307,0.364118,0.260144,0.192794,0.14812
1,0.801277,1.04281,1.17848,1.02819,0.75129,0.525975,0.370645,0.264257,0.194084,0.148471
0,0.681565,0.809905,0.870378,0.794115,0.636758,0.476802,0.350455,0.247161,0.18301,0.142093


,0,1,2,3,4,5,6,7,8,9
9,8,8,9,9,9,10,10,10,9,9
8,9,10,11,11,11,11,12,11,11,10
7,12,13,13,14,14,14,14,13,12,11
6,16,17,16,17,17,18,18,16,15,15
5,23,23,21,20,23,24,23,21,23,22
4,39,34,24,24,33,35,40,40,33,27
3,89,41,19,21,75,84,76,56,44,35
2,213,303,362,300,201,134,92,66,49,37
1,204,265,300,262,191,134,94,67,49,37
0,173,206,221,202,162,121,89,63,46,36


lwa: 0.70417209858352 ; sf: 0.12959918186818403


## Bilan

OK, là, si vous exécutez ce notebook vous vous rendrez compte que ça fait très très mal pour ce qui n'est qu'une pauvre pièce de 10x10 cellules avec un découpage en patchs de 0.5x0.5 (donc 16 rayons pour déterminer le facteur de forme entre 2 cellules).

Ce qui a été mis en place initialement :

- ne considérer que les murs verticaux comme obstacles
- établir une liste des obstacles potentiels en fonction du couloir entre les deux cellules Rect considérées

Constat pour une pièce 10x10, avec 4 murs et un plafond identique au sol :

- 5.3 millions de tests d'intersection au total
- dans la premières version une partie très importante du temps a été perdue dans les appels à la fonction numpy.cross() pour calculer le résultat du produit vectoriel entre 2 vecteurs ! Cette fonction a été remplacée par le code explicite correspondant à l'opération
- après ce changement, sur un pc "de bureau" j'obtiens 41 secondes de l'initialisation à la fin du calcul de facteur de forme. Sur ces 41 secondes, 40.3 sont dédiées au facteur de forme... dont 33 pour hasOcclusion et 30 juste pour le test d'intersection rayon / triangle
- le calcul des facteurs de forme représente 97.95 du temps d'exécution du code complet
- Note: impossible sur un vieux eee avec un Atom N270 à 1.60Ghz il m'a fallu 9 minutes pour calculer les facteurs de forme dans ce notebook !
- il y a une dégradation significative des performances entre python2 et python3 (sur une machine de bureau presque on passe de 29 secondes en v2 à 41 en v3) !

## Et Maintenant ?

Pour clarifier la situation et savoir si cette technique pouvait toujours être envisagée dans Space Banner via une librairie externe, une partie en Cython, ... Le code a été porté en C avec quelques adaptations :

- seules les listes d'obstacles dans findObstacles sont dynamiques via une bête listée chainée non optimisée (pas de pool d'objets)
- toutes les autres structures ont une allocation de mémoire statique ! Note: **pour ce test**,  toutes ces structures vont donc joyeusement consommer de l'espace dans la pile alors que celui-ci ests limité à 8Mo le linux de test. Avec une pièce de 10x10 ça fait 550Ko environ ce qui est très loin du compte. Mais avec 20x20 c'est 6.4Mo qui sont réclamées... Evidemment, c'est une très mauvaise idée mais pour ce POC, ça passe.
- les fonctions de calcul de norme d'un vecteur, de normalisation, de produit scalaire et vectoriel et de calcul du déterminant d'une matrice 3x3 sont traitées par l'enchainement des opérations correspondantes et pas par une librairie

### Les chiffres

- de 9 minutes en Python sur le eee à 1.5 secondes en C sans optimisation du compilateur, à 1.09 secondes avec -O2 et 0.88 avc -Ofast (mais c'est en général déconseillé)
- sur le PC de bureau de 42 secondes en Python à 0.17 secondes en C sans optimisation du compilateur, à 0.076" avec -O2

### Bilan

Les temps obtenus en C indiquent que la méthode peut être retenue mais qu'elle doit être externalisée via :

- une communication Python / C
- l'utilisation de [Cython](https://cython.org/)

## Et Maintenant ?

Pour récapituler on a :

- une technique pour calculer les facteurs de forme de manière relativement performante quand des obstacles sont présents
- le moyen de calculer l'exitance de chaque surface d'une scène en fonction des sources lumineuses présentes
- le moyen de convertir cette exitance en couleur RGB via un _tone mapping operator_

Techniquement, il faudrait valider la communication Python / C et tester les résultats obtenus avec Cython.

Se reporter à la page [TECH vision et éclairage](https://github.com/hern42/SpaceBanner/wiki/TECH-vision-et-eclairage) du wiki pour la liste complète des actions à mener pour boucler ce POC.